In [1]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib as mpl, matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import classification_report
import warnings
from imblearn.over_sampling import SMOTE
warnings.filterwarnings('ignore')

In [2]:
DWConnect = pyodbc.connect('Driver={SQL Server};'
                      'Server=SAMET;'
                      'Database=Projet3;'
                      'Trusted_Connection=yes;')
cursor_Dim = DWConnect.cursor()

In [3]:
SQL_Query = pd.read_sql_query('''select * FROM [dbo].[DimConditions]''', DWConnect)
DimConditions = pd.DataFrame(SQL_Query)
SQL_Query = pd.read_sql_query('''select * FROM [dbo].[DimTypeAccidentSeverity]''', DWConnect)
DimTypeAccidentSeverity = pd.DataFrame(SQL_Query)
SQL_Query = pd.read_sql_query('''select * FROM [dbo].[FactAccident]''', DWConnect)
FactAccident = pd.DataFrame(SQL_Query)

In [4]:
def decomposefact(fact,liste_1,liste_2):
    for i,j in enumerate(liste_1):
        fact= pd.merge(fact,j,on=liste_2[i])
    return fact

In [5]:
listetable=[DimConditions,DimTypeAccidentSeverity]
listeid=['code_condition','code_severity']

In [6]:
Accidents=decomposefact(FactAccident,listetable,listeid)

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [8]:
Accidents_class = Accidents
Accidents_class=Accidents_class[[ 'Light_Conditions'
      ,'Weather_Conditions'
      ,'Road_Surface_Conditions'
      ,'Special_Conditions_at_Site','Number_of_Vehicles','Accident_Severity']]
le = LabelEncoder()

for i in Accidents_class.columns:
    if i != 'Accident_Severity':
        Accidents_class[i] = le.fit_transform(Accidents_class[i])

In [9]:
train_frame=Accidents_class
test_frame=Accidents_class
X = train_frame.drop("Accident_Severity",axis=1)
#X_test = test_frame.drop("Accident_Severity",axis=1)
Y = train_frame["Accident_Severity"]
#y_test = test_frame["Accident_Severity"]

In [28]:
#model = RandomForestClassifier()
#model.fit(X.values,Y)
# Testing the model
#y_pred=model.predict(X_test.values)
#accuracy_score(y_test, y_pred)


In [11]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split



# Oversample the minority class using SMOTE
smote = SMOTE()
X_oversampled, y_oversampled = smote.fit_resample(X, Y)

# Split the oversampled dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_oversampled, y_oversampled, test_size=0.2)

# Train a Random Forest model on the training set
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Use the model to make predictions on the test set
predictions = model.predict(X_test)

In [12]:
# Model evaluvation
accuracy_score(y_test, predictions)

0.4460662542053349

In [13]:
from joblib import  dump

In [14]:
dump(model,'./../savedModeles/model.joblib')

['./../savedModeles/model.joblib']